# Visualize a geospatial Delta Lake table with Lonboard

::: {.callout-note}

The below example uses a column where the geometries are stored as WKB in a BINARY column. If you have a table using the newer GEOMETRY or GEOGRAPHY types, you can use the same logic but first use `st_asbinary()`, as well as `st_transform(..., ..., 4326)` as needed.

:::

[Lonboard](https://developmentseed.org/lonboard/latest/) is an excellent tool to visualize geospatial data via Arrow or [DuckDB](../duckdb-on-databricks/DuckDB_on_Databricks.ipynb). While it cannot visualize as large datasets as you can with using [streaming Flatgeobuf or PMTiles](../other_formats/export.ipynb), it can go far -- theoretically your browser's memory (and your network) is the limit. On Databricks, however, there is a quite frugal limit for the sizes of widgets, so we'll have to limit the sample dataset a bit more.

In [0]:
%pip install duckdb lonboard shapely --quiet

In [0]:
LIMIT = 100_000
WKB_COL = 'geometry'

In [0]:
import duckdb
from lonboard import viz

duckdb.sql("install spatial; load spatial")

dfa = (
    spark.table("carto_overture_maps_divisions.carto.division_area")
    .selectExpr(WKB_COL, "names:primary::string as name")
    .where(
        """country='NL' and subtype='county'"""
    )
    .limit(LIMIT)
    .toArrow()
)

if dfa.num_rows == LIMIT:
    print("WARNING: LIMIT reached, consider using a tighter `where()` filter")

query = duckdb.sql(
    f"select * replace (st_geomfromwkb({WKB_COL}) as {WKB_COL}) from dfa"
)
m = viz(query)
m.as_html()


![lonboard](img/lonboard.png)

If all went well and you see data on the map: note that you can not just zoom and pan, but can also click on a polygon to see the value of the non-geometry column(s).

::: {.callout-note}

If you receive the error:

```
Command result size exceeds limit: Exceeded XXXX bytes (current = XXXX)
```

Then you need to reduce your `LIMIT` or tighten your filter. Also consider filtering only for the columns you actually need (besides the geometry column, of course).

:::